**Aim**
* Predict if the student is giong to pass or fail

In [ ]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as sm

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import accuracy_score,roc_auc_score,confusion_matrix

In [ ]:
#Reading data
data = pd.read_csv('../input/StudentsPerformance.csv')

In [ ]:
data.head()

## Data Pre-Processing

In [ ]:
#Lets clean column names so it will be easier to work
data.columns = data.columns.str.strip().str.lower().str.replace(' ','_').str.replace('/','_')

In [ ]:
data.info()

**As we can see there are no missing values**

**Lets see the distrubution of maath score,reading score, writing score to spot any outliers**

In [ ]:
sns.pairplot(data.loc[:,['math_score','reading_score','writing_score']],height=4,aspect=1)

**We dont have any outliers in this data**

In [ ]:
data.describe()

## Feature Engineering 

I am going to create 4 new features from given data

* Total marks - Combining math score, reading score, writing score
* Result - Weather the student is pass or fail in the exam
* Percentage - Percentage of the student in the exam
* Performance - How well a student performed based on the grading system

For better analysis and prediction

In [ ]:
#Lets create few new features
#Total marks
data['total_marks'] = data.math_score + data.reading_score + data.writing_score
#Result
data['result'] = np.where(data['total_marks']<120, 'F', 'P')
#Percentage
data['percent'] = data.total_marks/3
#Grading - How good the student performed based on percentage
data['performance'] = pd.cut(data.percent,bins=[0,39,60,70,80,90,100],labels=['F','C','B','B+','A','A+'])
data.head()

In [ ]:
data.info()

## EDA

Let's check the pass percentage between male and female students 

### result

In [ ]:
labels = ['male students pass','female students pass']
sizes = [data[data.result == 'P']['gender'].value_counts()[1],
         data[data.result == 'P']['gender'].value_counts()[0]]
fig1, ax1 = plt.subplots(figsize=(20,10))
ax1.pie(sizes, labels=labels, autopct='%1.1f%%', shadow=True)
ax1.axis('equal')
plt.show()

Female students pass percentage is 4% higher than male students pass percentage
* male students - 48.2%
* female students - 51.8%

### race_ethnicity

In [ ]:
fig1, ax1 = plt.subplots(figsize=(20,10))
ax1.pie(data[data.result == 'P']['race_ethnicity'].value_counts(),explode=(.1,.1,0,0,0),labels=['group C','group D','group B','group E','group A'],autopct='%1.1f%%',shadow=True,startangle=90)
ax1.axis('equal')
plt.show()

So the students from group-C and group-D have good performance when compared to other groups

### parental_level_of_education

In [ ]:
fig1, ax1 = plt.subplots(figsize=(20,10))
sns.countplot(data.parental_level_of_education,hue=data.result,ax=ax1)

**I am going to create new column based on parental_level_of_education for better analysis**

In [ ]:
degree_data = {
    'high school' :  'basic_education',
    'some high school' :  'basic_education',
    "associate's degree" : 'good education',
    "some college" : 'good education',
    "bachelor's degree" : 'high level education',
    "master's degree" : 'high level education'
}

In [ ]:
data['parents_education'] = data.parental_level_of_education.map(degree_data)

In [ ]:
fig1, ax1 = plt.subplots(figsize=(16,8))
sns.countplot(data.parents_education,ax=ax1)
sns.despine(left=True,bottom=True)
sns.set_style({'ytick.left': False,'xtick.top': False})

In [ ]:
data[data.parents_education == 'basic_education']['result'].value_counts()

In [ ]:
fig = plt.figure(figsize=(20,10))
ax1 = fig.add_subplot(1,3,1)
ax2 = fig.add_subplot(1,3,2)
ax3 = fig.add_subplot(1,3,3)
ax1.pie(data[data.parents_education == 'basic_education']['result'].value_counts(),labels=['Pass','Fail'],autopct='%1.1f%%',shadow=True,startangle=90)
ax1.axis('equal')
ax1.set_title('Basic Education')
ax2.pie(data[data.parents_education == 'good education']['result'].value_counts(),labels=['Pass','Fail'],autopct='%1.1f%%',shadow=True,startangle=90)
ax2.axis('equal')
ax2.set_title('Good Education')
ax3.pie(data[data.parents_education == 'high level education']['result'].value_counts(),labels=['Pass','Fail'],autopct='%1.1f%%',shadow=True,startangle=90)
ax3.axis('equal')
ax3.set_title('High level Education')
plt.show()

So the students whose parents have basic education have less pass percent when compared to the parents with good and high-level education
* Students of parents with basic education pass percent - 94.7%
* Students of parents with good education pass percent - 98.2%
* Students of parents with high-level education pass percent - 98.9%

### Lunch


let's see if lunch has any impact on student performance

In [ ]:
fig1, ax1 = plt.subplots(figsize=(16,8))
sns.countplot(data.lunch,hue=data.result,ax=ax1)
sns.despine(left=True,bottom=True)
sns.set_style({'ytick.left': False,'xtick.top': False})

**So students with free or reduced lunch might not perform well in exams**

### test_preparation_course

In [ ]:
fig1, ax1 = plt.subplots(figsize=(16,8))
sns.countplot(data.test_preparation_course,ax=ax1)
sns.despine(left=True,bottom=True)
sns.set_style({'ytick.left': False,'xtick.top': False})

** More than 50% of students didnt complete test preparation but let's see if there is any difference in pass percent**

In [ ]:
fig = plt.figure(figsize=(20,10))
ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)
ax1.pie(data[data.test_preparation_course == 'none']['result'].value_counts(),labels=['Pass','Fail'],autopct='%1.1f%%',shadow=True,startangle=90)
ax1.axis('equal')
ax1.set_title('Test Preperation None')
ax2.pie(data[data.test_preparation_course == 'completed']['result'].value_counts(),labels=['Pass','Fail'],autopct='%1.1f%%',shadow=True,startangle=90)
ax2.axis('equal')
ax2.set_title('Test Preperation Completed')

**The students who completed the course have more pass percentage than students who didn't complete the course so test preparation course does have an impact on the student performance**

we can even analyze further and compare the independent performances in math, reading, writing but I am going to stop this analysis here

**Conclusion :**
The students with the following attribute are gonna perform better than other students
* race_ethinicity:        group c or group d
* parents _education:  Good education or high-level education. 
* lunch:                       Standard
* test_preperation:      Completed


## Prediction

**In this notebook i am going to predict**
- Weather student is going to pass or fail

## Data Pre-Processing


In [ ]:
data.head()

**I am going to drop all un-relevant features**

In [ ]:
data_mod = data.drop(['math_score','reading_score','writing_score','total_marks','percent','performance','parents_education'],axis=1)

In [ ]:
data_mod.head()

**I am gonna encode the categorical data**

In [ ]:
#Creating dummy variables
dum_r_e = pd.get_dummies(data_mod.race_ethnicity,prefix='re',dtype=int)
dum_p_l_e = pd.get_dummies(data_mod.parental_level_of_education,prefix='ple',dtype=int)

label_encoder = LabelEncoder()
data_mod.gender = label_encoder.fit_transform(data_mod.gender)
data_mod.lunch = label_encoder.fit_transform(data_mod.lunch)
data_mod.test_preparation_course = label_encoder.fit_transform(data_mod.test_preparation_course)
data_mod.result = label_encoder.fit_transform(data_mod.result)

data_mod = pd.concat([data_mod,dum_r_e,dum_p_l_e],axis=1)

data_mod = data_mod.drop(['race_ethnicity','parental_level_of_education'],axis=1)

data_mod.head()

In [ ]:
fig, ax = plt.subplots(figsize=(16,8))
sns.heatmap(data_mod.corr(),annot=True,cmap='Blues',ax=ax)

In [ ]:
data_mod.dtypes

In [ ]:
x = data_mod.drop(['result'],axis=1).values
y = data_mod.loc[:,'result'].values

In [ ]:
def backward_elimination(x_back,y,signifance_level=0.05):
    lenx = len(x_back[0])
    temp = np.zeros((1000,14)).astype(int)
    for i in range(0, lenx):
        rand_regressor_ols = sm.OLS(y, x_back).fit()
        maxPvalue = max(rand_regressor_ols.pvalues)
        adj_Rb = rand_regressor_ols.rsquared_adj.astype(float)
        if maxPvalue > signifance_level:
            for j in range(0, lenx - i):
                if (rand_regressor_ols.pvalues[j].astype(float) == maxPvalue):
                    temp[:,j] = x_back[:, j]
                    x_back = np.delete(x_back, j, 1)
                    tmp_regressor = sm.OLS(y, x_back).fit()
                    adj_Ra = tmp_regressor.rsquared_adj.astype(float)
                    if (adj_Rb >= adj_Ra):
                        x_rollback = np.hstack((x_back, temp[:,[0,j]]))
                        x_rollback = np.delete(x_rollback, j, 1)
                        print (rand_regressor_ols.summary())
                        return x_rollback
                    else:
                        continue
    rand_regressor_ols.summary()
    return x_back

In [ ]:
x = backward_elimination(x,y)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3,random_state = 1)

In [ ]:
params = {  "bootstrap":[True,False],
            "criterion":['entropy','gini'],
            "max_depth": [5,10,15],
            "max_features": ["log2", "sqrt"],
            "min_samples_split": [3,6,10],
            "min_samples_leaf": [1, 5],
            "n_estimators": [10,100,150]         
         }
rf_cla = RandomForestClassifier(random_state=1)
grid = GridSearchCV(rf_cla,param_grid=params,scoring='roc_auc',cv=10)
grid.fit(x_train,y_train)

In [ ]:
grid.best_params_

In [ ]:
y_pred = grid.best_estimator_.predict(x_test)
y_predtrain = grid.best_estimator_.predict(x_train)

In [ ]:
print('Accuracy Score of test set {0}'.format(accuracy_score(y_test,y_pred)))
print('Accuracy Score of training set {0}'.format(accuracy_score(y_train,y_predtrain)))